# `logistic-regression-pytorch`

Task: Fit a logistic regression model using PyTorch and fastai's Learner class.

## Setup

In [ ]:
!pip install -Uq fastbook

In [5]:
from fastai.tabular.all import *

In [4]:
penguins_csv = download_data('https://raw.githubusercontent.com/allisonhorst/palmerpenguins/master/inst/extdata/penguins.csv')
penguins = pd.read_csv(penguins_csv)
penguins.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007


In [8]:
len(penguins)

344

In [10]:
penguins = penguins.dropna()

In [11]:
len(penguins)

333

In [15]:
splitter = RandomSplitter(valid_pct=0.2, seed = 2021032)

In [25]:
dls = TabularDataLoaders.from_df(
    penguins,
    procs=[Categorify, Normalize],
    y_names="species",
#    cat_names=['island'],
    cont_names=['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g'],
    splitter=splitter,
    bs=10
)
dls.show_batch()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,species
0,38.200001,18.1,184.999999,3950.000005,Adelie
1,50.400001,15.7,222.000000,5750.000016,Gentoo
2,48.400002,14.6,213.000000,5849.999985,Gentoo
3,40.500000,18.9,180.000001,3950.000005,Adelie
4,39.200001,18.6,190.000000,4250.000000,Adelie
5,50.500000,15.9,222.000000,5549.999984,Gentoo
6,43.200001,18.5,192.000000,4099.999994,Adelie
7,49.599998,15.0,216.000001,4749.999994,Gentoo
8,37.299999,20.5,199.000000,3774.999988,Adelie
9,44.900002,13.3,213.000000,5099.999981,Gentoo


In [41]:
_, X_batch, y_batch = dls.one_batch()
y_batch = y_batch[:,0].long() # hack to get it in the right shape
X_batch

tensor([[ 1.3874,  1.6150, -0.5350, -0.6136],
        [ 0.8844,  1.2084,  0.6118, -0.3945],
        [-1.5932, -0.1131, -1.1800, -1.3963],
        [ 0.6236, -1.3329,  0.9701,  1.4214],
        [-0.1961,  0.7001, -0.6783, -0.2066],
        [-0.8481,  0.6493, -0.8217, -0.4571],
        [-1.5373,  0.3444, -0.4633, -1.0206],
        [-1.4069,  1.1068, -0.6067, -1.0206],
        [-0.8853,  0.3444, -0.9650, -1.2085],
        [ 0.6609, -1.0788,  0.9701,  1.7345]])

## Task

Fit a Logistic Regression model on the Palmer Penguins dataset.

## Solution


**Fill in the blanks to construct a `model`**:

```
model = nn.Linear(in_features=..., out_features=..., bias=...)
```

* For `in_features`, think about the shape of the data coming in.
* For `out_features`, think about how many logits we're going to need.

In [43]:
# your code here
model = nn.Linear(in_features=4, out_features=3, bias=True)

To check that we got it right, **call the `model` with the input data from one example batch**.

Let's get an example batch from the training set dataloader:

Now, give that batch to the model:

In [44]:
# your code here
model(X_batch)

tensor([[-1.4123, -0.7648,  0.9538],
        [-0.9624, -0.6809,  0.2113],
        [-0.8675,  0.3887,  0.8898],
        [ 1.0350, -0.8553, -0.4405],
        [-0.7891, -0.2374,  0.4773],
        [-0.8625,  0.0334,  0.4503],
        [-0.8586,  0.3071,  0.3403],
        [-1.2596,  0.2935,  0.3401],
        [-1.0112,  0.1125,  0.8678],
        [ 1.0253, -0.8884, -0.5973]], grad_fn=<AddmmBackward>)

In [45]:
model.weight.shape

torch.Size([3, 4])

In [46]:
model.bias.shape

torch.Size([3])

**Create a `loss_func` by instantiating an `nn.CrossEntropyLoss`.**

In [47]:
# your code here
loss_func = nn.CrossEntropyLoss()

**Evaluate the loss on the  `loss_func` on the example batch.** 

Note: PyTorch loss functions take inputs, then targets. `sklearn` loss functions (metrics) use the reverse order.

In [48]:
loss_func(model(X_batch), y_batch)

tensor(2.0000, grad_fn=<NllLossBackward>)

**Construct a `Learner`.**

* Use the `dataloaders`, `model`, and `loss_func` constructed above.
* Omit `opt_func` to use the default.
* Use `accuracy` as a metric.

In [57]:
class WrapTabular(nn.Module):
    def __init__(self, module): 
        super().__init__()
        self.module = module
    def forward(self, cats, conts): return self.module(conts)

In [58]:
learner = TabularLearner(
    dls=dls,
    model=WrapTabular(model),
    loss_func=loss_func,
    metrics=[accuracy],
)

**Train the model for 50 epochs**.

Use a learning rate of 1e-1.

Note: If you want to try alternative learning rates or other parameters here,
call `learner.reset_parameters()` before `fit`.

In [59]:
learner.fit(50, lr=1e-1)

epoch,train_loss,valid_loss,accuracy,time


RuntimeError: 1D target tensor expected, multi-target not supported

**Plot the training and validation loss.**

You can use the convenient `plot_loss` method of `learner.recorder`.

In [ ]:
learner.recorder.plot_loss()

**Make a prediction by hand**

In [ ]:
x = X_train[0:1]
model(x)

In [ ]:
y_train[0]

**Was this prediction correct?**

**Use `model.weight` and `model.bias` to make the same prediction**.

In [ ]:
x @ model.weight.T + model.bias

**Compute the predicted probabilities.**

The model outputs logits, so you'll need to use `softmax`. Be careful with dimensionality.

In [ ]:
model(x).softmax(dim=1)

In [ ]:
import sklearn.linear_model

In [ ]:
#clf = sklearn.linear_model.SGDClassifier(
#    loss='log', alpha=0.001, max_iter=100,
#    learning_rate='invscaling', eta0=1e2,
#    verbose=True
#)
clf = sklearn.linear_model.LogisticRegression(C=1.).fit(X[train_indices], y[train_indices])
clf.score(X[valid_indices], y[valid_indices])

In [ ]:
clf.intercept_

In [ ]:
clf.coef_

In [ ]:
model.weight.data.copy_(tensor(clf.coef_))

In [ ]:
model.bias.data.copy_(tensor(clf.intercept_))

In [ ]:
clf.t_

In [ ]:
clf.n_iter_

In [ ]:
alpha = 0.001
typw = np.sqrt(1.0 / np.sqrt(alpha))
# computing eta0, the initial learning rate
initial_eta0 = typw / max(1.0, clf.loss_function_.dloss(-typw, 1.0))
# initialize t such that eta at first sample equals eta0
optimal_init = 1.0 / (initial_eta0 * alpha)

t = np.arange(5400)
eta = 1.0 / (alpha * (optimal_init + t - 1))
plt.plot(t / len(train_indices), eta)

In [ ]:
sklearn.metrics.log_loss(y[valid_indices], clf.predict_proba(X[valid_indices]))

In [ ]:
sklearn.metrics.log_loss

## Analysis